# L1 Regularization(Lasso)

In [1]:
from sklearn.linear_model import Lasso
import numpy as np

# Step 1: 模擬數據集
# 生成 100 個樣本，每個樣本有 5 個特徵
X = np.random.randn(100, 5)  # 樣本特徵矩陣 (100x5)
# 生成目標變數 y，其中僅第一個特徵對 y 有顯著貢獻
# y = 3 * 第一個特徵 + 隨機噪聲
y = X[:, 0] * 3 + np.random.randn(100)  # 增加隨機噪聲來模擬實際數據

# Step 2: 初始化並訓練 Lasso 模型
# alpha 是正則化參數，用來控制懲罰項的強度
lasso = Lasso(alpha=0.1)
lasso.fit(X, y)

# Step 3: 輸出結果
# 查看 Lasso 模型學習到的係數 (coefficients)
# 非重要特徵的係數可能被壓縮為 0
print("Coefficients:", lasso.coef_)
# 截距項，表示當所有特徵值為 0 時，目標變數的預測值
print("Intercept:", lasso.intercept_)

Coefficients: [ 2.92335649 -0.         -0.         -0.          0.10261669]
Intercept: 0.13717627314517145


# L2 Regularization(Ridge)

In [2]:
from sklearn.linear_model import Ridge
import numpy as np

# Step 1: 模擬數據集
# 生成 100 個樣本，每個樣本有 5 個特徵
X = np.random.randn(100, 5)  # 樣本特徵矩陣 (100x5)
# 生成目標變數 y，其中第一和第二個特徵對 y 有顯著貢獻
# y = 3 * 第一個特徵 - 2 * 第二個特徵 + 隨機噪聲
y = X[:, 0] * 3 + X[:, 1] * -2 + np.random.randn(100)  # 增加隨機噪聲來模擬實際數據

# Step 2: 初始化並訓練 Ridge 模型
# alpha 是正則化參數，用來控制懲罰項的強度
ridge = Ridge(alpha=0.1)  # alpha 越大，正則化懲罰越強
ridge.fit(X, y)

# Step 3: 輸出結果
# 查看 Ridge 模型學習到的係數 (coefficients)
print("Coefficients:", ridge.coef_)
# 截距項，表示當所有特徵值為 0 時，目標變數的預測值
print("Intercept:", ridge.intercept_)

Coefficients: [ 2.99506969e+00 -2.01985894e+00 -7.25396330e-02 -5.88714865e-02
  1.98790431e-03]
Intercept: 0.0718301095239695


# Hyperparameter Tuning
Ideally we want a machine learning model to have low bias and low variance
bias-variance tradeoff: rying to minimize bias and variance simultaneously is a bit of a conundrum as lowering one raises the other

# GridSearchCV
GridSearchCV 是 Scikit-learn 中的一個工具，用於進行超參數的搜尋與交叉驗證。它的作用是遍歷指定的超參數組合，並通過交叉驗證來尋找最佳的超參數配置。這樣可以幫助我們選擇出最合適的模型，從而提高模型的預測精度。

In [3]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Step 1: 載入乳腺癌數據集
# `load_breast_cancer` 載入了關於乳腺癌的數據集
cancer = load_breast_cancer()

# Step 2: 將數據集分割成訓練集和測試集
# 訓練集和測試集的分配比率為 75% 訓練集，25% 測試集
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=19)

# Step 3: 初始化邏輯回歸模型和超參數範圍
# `solver='liblinear'` 適用於小型數據集的求解器，`max_iter=1000` 設定最大迭代次數
lr = LogisticRegression(solver='liblinear', max_iter=1000)

# Step 4: 檢查 Logistic Regression 模型的默認參數
# 顯示 Logistic Regression 模型的所有超參數設置
print("Logistic Regression parameters:", lr.get_params())

# Step 5: 設定 GridSearchCV 進行超參數搜索
# `penalty` 是正則化方法，可以選擇 'l1' 或 'l2'，`C` 是正則化強度的反比，數字越大正則化越弱
parameters = {'penalty': ['l1', 'l2'], 'C': [1, 10, 100]}

# Step 6: 設定 GridSearchCV 來進行超參數搜尋
# `GridSearchCV` 用來進行超參數的遍歷搜尋，`cv=5` 表示使用 5 折交叉驗證
clf = GridSearchCV(lr, parameters, cv=5)

# Step 7: 在訓練數據上擬合 GridSearchCV 並找到最佳模型
clf.fit(X_train, y_train)
best_model = clf.best_estimator_  # 提取最佳模型
print("Best model:", best_model)  # 顯示最佳模型
print("Best parameters:", clf.best_params_)  # 顯示最佳超參數組合

# Step 8: 計算最佳模型在訓練集和測試集上的表現
best_score = clf.best_score_  # 最佳模型在交叉驗證中的平均得分
test_score = clf.score(X_test, y_test)  # 測試集的得分
print("Best score (train):", best_score)  # 輸出最佳得分（訓練集）
print("Test score:", test_score)  # 輸出測試集得分

# Step 9: 查看 GridSearchCV 搜索過程中的結果
# 提取所有的超參數組合及其對應的測試分數
hyperparameter_grid = pd.DataFrame(clf.cv_results_['params'])  # 提取超參數組合
grid_scores = pd.DataFrame(clf.cv_results_['mean_test_score'], columns=['score'])  # 提取平均測試分數

# 合併超參數和對應的分數，顯示每個超參數組合的結果
df = pd.concat([hyperparameter_grid, grid_scores], axis=1)
print("Grid search results:")
print(df)


Logistic Regression parameters: {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'liblinear', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Best model: LogisticRegression(C=10, max_iter=1000, penalty='l1', solver='liblinear')
Best parameters: {'C': 10, 'penalty': 'l1'}
Best score (train): 0.9671135430916552
Test score: 0.951048951048951
Grid search results:
     C penalty     score
0    1      l1  0.955349
1    1      l2  0.952996
2   10      l1  0.967114
3   10      l2  0.957702
4  100      l1  0.957674
5  100      l2  0.962380


# RandomizedSearchCV

In [4]:
import pandas as pd
from sklearn.datasets import load_breast_cancer  # 載入乳腺癌數據集
from sklearn.model_selection import train_test_split  # 載入訓練集與測試集的分割方法
from sklearn.linear_model import LogisticRegression  # 載入邏輯回歸模型
from sklearn.model_selection import RandomizedSearchCV  # 載入隨機搜索CV
from scipy.stats import uniform  # 載入uniform分佈，用於隨機抽樣

# 載入乳腺癌數據集
cancer = load_breast_cancer()

# 將數據集分割為訓練集和測試集，80% 用於訓練，20% 用於測試
X = cancer.data  # 特徵數據
y = cancer.target  # 標籤數據
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 進行分割

# 定義要隨機搜索的超參數範圍
# penalty 參數可以選擇 'l1' 或 'l2'，C 是正則化的強度
distributions = {'penalty': ['l1', 'l2'], 'C': uniform(loc=0, scale=100)}  # C 在 0 到 100 之間均勻分佈

# 初始化邏輯回歸模型，solver 設為 'liblinear' 以適應小型數據集，max_iter 設為 1000 防止迭代過少
lr = LogisticRegression(solver='liblinear', max_iter=1000)

# 初始化隨機搜索CV，使用定義的超參數範圍，n_iter 設為 8 代表隨機搜索 8 次
clf = RandomizedSearchCV(lr, distributions, n_iter=8, random_state=42)

# 進行隨機搜索並擬合訓練數據，找到最佳模型
clf.fit(X_train, y_train)
best_model = clf.best_estimator_  # 提取最佳模型
print("Best Model:", best_model)
print("Best Hyperparameters:", clf.best_params_)

# 計算最佳模型在訓練集和測試集上的得分
best_score = clf.best_score_  # 最佳訓練得分
test_score = clf.score(X_test, y_test)  # 測試集得分
print("Best Training Score:", best_score)
print("Test Score:", test_score)

# 查看隨機搜索的結果，提取所有的超參數組合及其對應的測試得分
hyperparameter_values = pd.DataFrame(clf.cv_results_['params'])  # 提取超參數組合
randomsearch_scores = pd.DataFrame(clf.cv_results_['mean_test_score'], columns=['score'])  # 提取平均測試分數

# 合併超參數組合和分數，顯示每個超參數組合的結果
df = pd.concat([hyperparameter_values, randomsearch_scores], axis=1)
print(df)

# 額外檢查 C 參數的隨機抽樣結果，查看其分佈情況
first_draw = distributions['C'].rvs(10)  # 隨機抽取 10 個 C 的值
second_draw = distributions['C'].rvs(10)  # 再隨機抽取 10 個 C 的值
print("First Draw:", first_draw)
print("Second Draw:", second_draw)

Best Model: LogisticRegression(C=59.86584841970366, max_iter=1000, penalty='l1',
                   solver='liblinear')
Best Hyperparameters: {'C': 59.86584841970366, 'penalty': 'l1'}
Best Training Score: 0.9692307692307693
Test Score: 0.9824561403508771
           C penalty     score
0  37.454012      l1  0.967033
1  18.343479      l2  0.960440
2  59.865848      l1  0.969231
3  44.583275      l1  0.964835
4   5.808361      l2  0.960440
5  33.370861      l2  0.962637
6  70.807258      l2  0.962637
7   5.641158      l2  0.958242
First Draw: [98.99070216 35.15903074 85.29442762 39.32198395 81.63837489 68.27968736
 46.83954012 12.53955994 24.19485092 89.81103592]
Second Draw: [71.22041511 95.75982245  2.32827697 54.58696908  9.22251691 42.69351073
 41.50600743 67.22382074 93.67144686 26.84879395]


In [5]:
import datetime

current_date = datetime.datetime.now().strftime("%Y年%m月%d日")
print(f"更新日期: {current_date}")

更新日期: 2024年12月01日
